In [5]:
import heapq
import tkinter as tk

class Autobot:
    def __init__(self, start, goal, id):
        self.start = start
        self.goal = goal
        self.current_pos = start
        self.id = id
        self.path = []
        self.total_commands = 0
        self.total_time = 0
        self.direction = 0  # 0=up, 1=right, 2=down, 3=left
        self.commands = []
        self.waiting = False  # Initialize waiting attribute

    def forward(self):
        """ Move forward in the direction the bot is facing. """
        if self.direction == 0:  # Up
            return (self.current_pos[0] - 1, self.current_pos[1])
        elif self.direction == 1:  # Right
            return (self.current_pos[0], self.current_pos[1] + 1)
        elif self.direction == 2:  # Down
            return (self.current_pos[0] + 1, self.current_pos[1])
        elif self.direction == 3:  # Left
            return (self.current_pos[0], self.current_pos[1] - 1)

    def left(self):
        """ Turn left (90 degrees). """
        self.direction = (self.direction - 1) % 4
        self.commands.append("Left")
        self.total_commands += 1

    def right(self):
        """ Turn right (90 degrees). """
        self.direction = (self.direction + 1) % 4
        self.commands.append("Right")
        self.total_commands += 1

    def wait(self):
        """ Wait for 1 unit of time. """
        self.waiting = True
        self.commands.append("Wait")
        self.total_commands += 1


def is_valid(pos, grid):
    """ Check if the position is within bounds and not an obstacle. """
    x, y = pos
    return 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == '.'


def heuristic(a, b):
    """ Heuristic for A* (Manhattan distance). """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def a_star(grid, start, goal):
    """ A* Algorithm for pathfinding. """
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        _, current = heapq.heappop(open_set)

        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path

        for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if is_valid(neighbor, grid):
                tentative_g_score = g_score[current] + 1

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []  # No path found


class AutobotGUI:
    def __init__(self, root, grid, autobots):
        self.root = root
        self.grid = grid
        self.autobots = autobots
        self.canvas = tk.Canvas(self.root, width=500, height=500)
        self.canvas.pack()
        self.cell_size = 100 // max(len(grid), len(grid[0]))  # Cell size based on grid size
        self.draw_grid()
        self.start_button = tk.Button(self.root, text="Start Simulation", command=self.start_simulation)
        self.start_button.pack()
        self.simulation_step = 0  # Track the step of the simulation
        self.colors = ['blue', 'red', 'green', 'orange', 'purple']  # Add more colors as needed
    
    def draw_grid(self):
        rows = len(self.grid)
        cols = len(self.grid[0])
        for i in range(rows):
            for j in range(cols):
                color = "white"
                if self.grid[i][j] == "X":
                    color = "black"
                self.canvas.create_rectangle(j * self.cell_size, i * self.cell_size,
                                             (j + 1) * self.cell_size, (i + 1) * self.cell_size, fill=color)

    def draw_autobots(self):
        for bot in self.autobots:
            x, y = bot.current_pos
            color = self.colors[bot.id % len(self.colors)]  # Assign color based on bot id
            self.canvas.create_oval(y * self.cell_size, x * self.cell_size,
                                    (y + 1) * self.cell_size, (x + 1) * self.cell_size, fill=color)

    def start_simulation(self):
        self.simulation_step = 0  # Reset the simulation step
        self.simulate_autobots()

    def simulate_autobots(self):
        # Move both autobots simultaneously
        if self.simulation_step < max(len(bot.path) for bot in self.autobots):
            for bot in self.autobots:
                if self.simulation_step < len(bot.path):
                    bot.current_pos = bot.path[self.simulation_step]  # Update position of bot
            self.canvas.delete("all")
            self.draw_grid()
            self.draw_autobots()
            self.simulation_step += 1
            self.root.after(500, self.simulate_autobots)  # Schedule the next step after 500ms
        else:
            print("Simulation Complete")

    def update(self):
        self.canvas.update()


def main():
    root = tk.Tk()
    root.title("Autobot Navigation System")
    
    # Define grid (5x5 for simplicity)
    grid = [['.' for _ in range(5)] for _ in range(5)]
    grid[0][3] = 'X'
    grid[1][1] = 'X'
    grid[2][2] = 'X'
    grid[4][1] = 'X'
    
    # Create autobots
    autobots = []
    autobot1 = Autobot((0, 0), (2, 1), 0)
    autobot1.path = a_star(grid, (0, 0), (2, 1))
    autobots.append(autobot1)
    
    autobot2 = Autobot((4, 0), (2, 1), 1)
    autobot2.path = a_star(grid, (4, 0), (2, 1))
    autobots.append(autobot2)
    
    # Initialize GUI
    app = AutobotGUI(root, grid, autobots)
    
    root.mainloop()


if __name__ == "__main__":
    main()


Simulation Complete
